# Object-Detection using EfficientDet-D2 architecture

In [2]:
import os

os.makedirs('eagleview/config', exist_ok=True)
os.makedirs('eagleview/data', exist_ok=True)
os.makedirs('eagleview/data/processed', exist_ok=True)
os.makedirs('eagleview/data/raw', exist_ok=True)
os.makedirs('eagleview/logs', exist_ok=True)
os.makedirs('eagleview/notebooks', exist_ok=True)
os.makedirs('eagleview/scripts', exist_ok=True)

os.makedirs('eagleview/checkpoints', exist_ok=True)
os.makedirs('eagleview/checkpoints/pretrained', exist_ok=True)
os.makedirs('eagleview/checkpoints/finetuning', exist_ok=True)

os.makedirs('eagleview/results', exist_ok=True)

In [3]:
parent_dir = './eagleview'
raw_data_path = os.path.join(parent_dir, 'data/raw')
processed_data_path = os.path.join(parent_dir, 'data/processed')
checkpoint_dir = os.path.join(parent_dir, 'checkpoints')

In [4]:
if not os.path.exists(raw_data_path + '/trainval.tar.gz'):
    os.system('wget https://evp-ml-data.s3.us-east-2.amazonaws.com/ml-interview/openimages-personcar/trainval.tar.gz -P ' + raw_data_path )
    os.system('tar xf '+raw_data_path+'/trainval.tar.gz -C '+raw_data_path)
    os.system('rm '+raw_data_path+'/trainval.tar.gz')

In [5]:
if not os.path.exists(checkpoint_dir+'/pretrained/efficientdet-d2.tar.gz'):
    pretrained_dir = checkpoint_dir+'/pretrained'
    os.system('wget  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-d2.tar.gz -P ' + pretrained_dir)
    os.system('tar xf '+pretrained_dir+'/efficientdet-d2.tar.gz -C '+pretrained_dir)
    os.system('rm '+pretrained_dir+'/efficientdet-d2.tar.gz')

os.system('git clone --depth 1 https://github.com/google/automl ' + parent_dir+'/scripts')

0

In [6]:
!pip install -U PyYAML

!pip install pycocotools
!pip install --user --upgrade tensorflow-model-optimization

     |████████████████████████████████| 276kB 14.9MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=960907ca9cbd77b91e9096cb83c07de644fe966507d7fee69efb92386a2b5272
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 174kB 13.5MB/s 


In [7]:
import numpy as np
import pandas as pd
import glob
import json
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import io
import os
from PIL import Image
import yaml


# Data Preparation

In [9]:
with open(raw_data_path+'/trainval/annotations/bbox-annotations.json') as json_data:
    data = json.load(json_data)

In [10]:
df_images = pd.DataFrame(data['images'])
df_images['file_path'] = raw_data_path + '/trainval/images/' + df_images['file_name']

In [11]:
df_images.head()

,file_name,width,height,id,license,file_path
0,image_000000001.jpg,1024,768,0,1,./eagleview/data/raw/trainval/images/image_000...
1,image_000000002.jpg,1024,576,1,1,./eagleview/data/raw/trainval/images/image_000...
2,image_000000003.jpg,1024,683,2,1,./eagleview/data/raw/trainval/images/image_000...
3,image_000000004.jpg,1024,675,3,1,./eagleview/data/raw/trainval/images/image_000...
4,image_000000005.jpg,1024,926,4,1,./eagleview/data/raw/trainval/images/image_000...


In [12]:
df_annotations = pd.DataFrame(data['annotations'])
df_annotations.head()

,category_id,image_id,segmentation,iscrowd,bbox,area,id,license
0,1,0,[],0,"[846, 145, 146, 477]",0.088554,0,2
1,1,0,[],0,"[848, 216, 175, 551]",0.122611,1,2
2,2,0,[],0,"[74, 159, 75, 81]",0.007725,2,2
3,2,0,[],0,"[153, 124, 658, 643]",0.537992,3,2
4,1,1,[],0,"[488, 281, 28, 71]",0.003370,4,2


In [13]:
df = pd.merge(df_images, df_annotations, left_on='id', right_on='image_id')
df.head()

,file_name,width,height,id_x,license_x,file_path,category_id,image_id,segmentation,iscrowd,bbox,area,id_y,license_y
0,image_000000001.jpg,1024,768,0,1,./eagleview/data/raw/trainval/images/image_000...,1,0,[],0,"[846, 145, 146, 477]",0.088554,0,2
1,image_000000001.jpg,1024,768,0,1,./eagleview/data/raw/trainval/images/image_000...,1,0,[],0,"[848, 216, 175, 551]",0.122611,1,2
2,image_000000001.jpg,1024,768,0,1,./eagleview/data/raw/trainval/images/image_000...,2,0,[],0,"[74, 159, 75, 81]",0.007725,2,2
3,image_000000001.jpg,1024,768,0,1,./eagleview/data/raw/trainval/images/image_000...,2,0,[],0,"[153, 124, 658, 643]",0.537992,3,2
4,image_000000002.jpg,1024,576,1,1,./eagleview/data/raw/trainval/images/image_000...,1,1,[],0,"[488, 281, 28, 71]",0.003370,4,2


In [14]:
df.describe()

,width,height,id_x,license_x,category_id,image_id,iscrowd,area,id_y,license_y
count,16772.000000,16772.000000,16772.000000,16772.0,16772.00000,16772.000000,16772.0,16772.000000,16772.000000,16772.0
mean,999.082220,742.922371,1090.303363,1.0,1.35607,1090.303363,0.0,0.065363,8385.500000,2.0
std,102.926163,125.908542,642.705912,0.0,0.47885,642.705912,0.0,0.157283,4841.803693,0.0
min,575.000000,282.000000,0.000000,1.0,1.00000,0.000000,0.0,0.000000,0.000000,2.0
25%,1024.000000,683.000000,523.000000,1.0,1.00000,523.000000,0.0,0.002963,4192.750000,2.0
50%,1024.000000,696.000000,1082.000000,1.0,1.00000,1082.000000,0.0,0.010517,8385.500000,2.0
75%,1024.000000,768.000000,1630.000000,1.0,2.00000,1630.000000,0.0,0.038902,12578.250000,2.0
max,4496.000000,3000.000000,2238.000000,1.0,2.00000,2238.000000,0.0,0.998048,16771.000000,2.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16772 entries, 0 to 16771
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   file_name     16772 non-null  object 
 1   width         16772 non-null  int64  
 2   height        16772 non-null  int64  
 3   id_x          16772 non-null  int64  
 4   license_x     16772 non-null  int64  
 5   file_path     16772 non-null  object 
 6   category_id   16772 non-null  int64  
 7   image_id      16772 non-null  int64  
 8   segmentation  16772 non-null  object 
 9   iscrowd       16772 non-null  int64  
 10  bbox          16772 non-null  object 
 11  area          16772 non-null  float64
 12  id_y          16772 non-null  int64  
 13  license_y     16772 non-null  int64  
dtypes: float64(1), int64(9), object(4)
memory usage: 1.9+ MB


In [16]:
#COCO bounding box format is [top left x position, top left y position, width, height].

In [19]:
def apply_transformation_to_df(row):
    bbox = row['bbox']
    top_left_x = bbox[0]
    top_left_y = bbox[1]
    width = bbox[2]
    height = bbox[3]
    
    bottom_right_x = width + top_left_x
    bottom_right_y = height + top_left_y
    
    row['x_min'] = top_left_x
    row['y_min'] = top_left_y
    row['x_max'] = bottom_right_x
    row['y_max'] = bottom_right_y
    
    return row

In [20]:
df['x_min'] = 0
df['y_min'] = 0
df['x_max'] = 0
df['y_max'] = 0

In [21]:
df = df.apply(apply_transformation_to_df, axis=1)

In [22]:
df.head(2)

,file_name,width,height,id_x,license_x,file_path,category_id,image_id,segmentation,iscrowd,bbox,area,id_y,license_y,x_min,y_min,x_max,y_max
0,image_000000001.jpg,1024,768,0,1,./eagleview/data/raw/trainval/images/image_000...,1,0,[],0,"[846, 145, 146, 477]",0.088554,0,2,846,145,992,622
1,image_000000001.jpg,1024,768,0,1,./eagleview/data/raw/trainval/images/image_000...,1,0,[],0,"[848, 216, 175, 551]",0.122611,1,2,848,216,1023,767


# Creating TF Records

In [23]:
def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_list_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

## Train Test Split

In [24]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def class_int_to_text(val):
    if val==1:
        return 'Person'
    if val==2:
        return 'Car'

def create_tf_example(df, row):
    with tf.io.gfile.GFile(row.file_path, 'rb') as fid:
        encoded_jpg = fid.read()
        
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    
    width, height = image.size

    filename = row.file_name.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    
    for index, row in df.iterrows():
        xmins.append(row['x_min'] / width)
        xmaxs.append(row['x_max'] / width)
        ymins.append(row['y_min'] / height)
        ymaxs.append(row['y_max'] / height)
        classes.append(row['category_id'])
        classes_text.append(class_int_to_text(row['category_id']).encode('utf8'))
        

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(filename),
        'image/source_id':bytes_feature('0'.encode('utf8')),
        'image/encoded':bytes_feature(encoded_jpg),
        'image/format': bytes_feature(image_format),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text':bytes_list_feature(classes_text),
        'image/object/class/label':int64_list_feature(classes),
    }))
    return tf_example


def create_tf_record(df, subset, output_path):
    writer = tf.io.TFRecordWriter(output_path)
    
    for index, row in tqdm(subset.iterrows()):
        local_subset = df[df.file_name == row.file_name]
        tf_example = create_tf_example(local_subset, row)
        writer.write(tf_example.SerializeToString())
        
    writer.close()
    print('TFRecords created')


In [25]:
from sklearn.model_selection import train_test_split

In [26]:
from tqdm.notebook import tqdm

In [27]:
_X_train, _X_test = train_test_split(df['file_name'].unique(), test_size =0.1, random_state =42)
_X_train, _X_valid = train_test_split(_X_train, test_size =0.1, random_state =42)

In [28]:
X_train = df[df['file_name'].isin(_X_train)]
X_valid = df[df['file_name'].isin(_X_test)]
X_test = df[df['file_name'].isin(_X_valid)]

In [29]:
def check_leakage(df1, df2, col):
    unique_df1 = np.unique(df1[col])
    unique_df2 = np.unique(df2[col])
    
    for i in unique_df1:
        if i in unique_df2:
            return True
        
    return False

## Check Leakage

In [30]:
check_leakage(X_train, X_valid, 'file_name')

False

In [31]:

check_leakage(X_train, X_test, 'file_name')

False

In [32]:
if not os.path.exists(processed_data_path+'/test.record'):
    create_tf_record(df, X_test, processed_data_path+'/test.record')


TFRecords created


In [33]:
if not os.path.exists(processed_data_path+'/valid.record'):
    create_tf_record(df, X_valid, processed_data_path+'/valid.record')


TFRecords created


In [34]:
if not os.path.exists(processed_data_path+'/train.record'):
    create_tf_record(df, X_train, processed_data_path+'/train.record')


TFRecords created


# Start Training

In [35]:

dict_file = {'num_classes': 2,  'var_freeze_expr': '(efficientnet|fpn_cells|resample_p6)','label_map': {1: 'person', 2: 'car'}}

with open(parent_dir+'/config/config.yaml', 'w+') as file:
    documents = yaml.dump(dict_file, file)

In [79]:
train_command = 'python '+parent_dir+'/scripts/efficientdet/main.py --mode=train_and_eval \
 --train_file_pattern='+parent_dir+'/data/processed/train.record \
 --val_file_pattern='+parent_dir+'/data/processed/valid.record \
--model_name=efficientdet-d2 \
--model_dir='+parent_dir+'/checkpoints/finetuning \
--model_name=efficientdet-d2 \
--ckpt='+parent_dir+'/checkpoints/pretrained/efficientdet-d2/ \
--train_batch_size=16 \
--eval_batch_size=16 \
--num_epochs=10 \
--num_examples_per_epoch='+str(X_train.file_name.nunique())+' \
--eval_samples='+str(X_valid.file_name.nunique())+' \
--hparams='+parent_dir+'/config/config.yaml'

with open( parent_dir+'/scripts/train.sh', 'w+') as file:
    file.writelines(train_command)

In [80]:
!bash eagleview/scripts/train.sh 2>&1 | tee eagleview/logs/train_log.out

Streaming output truncated to the last 5000 lines.
I0116 18:26:01.368616 140116968068992 efficientdet_keras.py:749] fnode 6 : {'feat_level': 6, 'inputs_offsets': [3, 5, 10]}
I0116 18:26:01.369445 140116968068992 efficientdet_keras.py:749] fnode 7 : {'feat_level': 7, 'inputs_offsets': [4, 11]}
I0116 18:26:01.370815 140116968068992 efficientdet_keras.py:749] fnode 0 : {'feat_level': 6, 'inputs_offsets': [3, 4]}
I0116 18:26:01.371752 140116968068992 efficientdet_keras.py:749] fnode 1 : {'feat_level': 5, 'inputs_offsets': [2, 5]}
I0116 18:26:01.373521 140116968068992 efficientdet_keras.py:749] fnode 2 : {'feat_level': 4, 'inputs_offsets': [1, 6]}
I0116 18:26:01.374391 140116968068992 efficientdet_keras.py:749] fnode 3 : {'feat_level': 3, 'inputs_offsets': [0, 7]}
I0116 18:26:01.375274 140116968068992 efficientdet_keras.py:749] fnode 4 : {'feat_level': 4, 'inputs_offsets': [1, 7, 8]}
I0116 18:26:01.376068 140116968068992 efficientdet_keras.py:749] fnode 5 : {'feat_level': 5, 'inputs_offsets

In [85]:
!zip -r checkpoints.zip eagleview/checkpoints/finetuning/
from google.colab import files
files.download("checkpoints.zip")

  adding: eagleview/checkpoints/finetuning/ (stored 0%)
  adding: eagleview/checkpoints/finetuning/config.yaml (deflated 50%)
  adding: eagleview/checkpoints/finetuning/eval/ (stored 0%)
  adding: eagleview/checkpoints/finetuning/eval/events.out.tfevents.1610821559.5a006413a3ca (deflated 81%)
  adding: eagleview/checkpoints/finetuning/model.ckpt-906.index (deflated 76%)
  adding: eagleview/checkpoints/finetuning/backup/ (stored 0%)
  adding: eagleview/checkpoints/finetuning/backup/model.ckpt-113.meta (deflated 90%)
  adding: eagleview/checkpoints/finetuning/backup/best_eval.txt (deflated 50%)
  adding: eagleview/checkpoints/finetuning/backup/model.ckpt-113.data-00000-of-00001 (deflated 8%)
  adding: eagleview/checkpoints/finetuning/backup/model.ckpt-113.index (deflated 76%)
  adding: eagleview/checkpoints/finetuning/backup/checkpoint (deflated 42%)
  adding: eagleview/checkpoints/finetuning/model.ckpt-1006.meta (deflated 90%)
  adding: eagleview/checkpoints/finetuning/model.ckpt-1119.d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# <a>Inference</a>

In [102]:
from tqdm.notebook import tqdm

### Export Model

In [116]:
!python eagleview/scripts/efficientdet/model_inspect.py \
    --runmode=saved_model \
    --model_name=efficientdet-d2 \
    --ckpt_path=eagleview/checkpoints/finetuning \
    --saved_model_dir=eagleview/checkpoints/exported_model \
    --hparams=eagleview/config/config.yaml

2021-01-16 20:13:21.517482: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-16 20:13:23.714623: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-16 20:13:23.715629: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-16 20:13:23.750992: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-16 20:13:23.751609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2021-01-16 20:13:23.751650: I tensorflow/stream_executor/platform/default/dso_loade

### Run Inference

In [132]:
X_test_paths = X_test.file_path.unique()


for image in tqdm(X_test_paths):
    cmd= 'python eagleview/scripts/efficientdet/model_inspect.py \
--runmode=saved_model_infer \
--model_name=efficientdet-d2 \
--saved_model_dir=eagleview/checkpoints/exported_model \
--hparams=eagleview/config/config.yaml \
--input_image='+image+' \
--output_image_dir=eagleview/results'
    filename = image.split('/')[-1]
    cmd_2 = 'mv eagleview/results/0.jpg eagleview/results/'+filename
    

    os.system(cmd)
    os.system(cmd_2)



In [133]:
!zip -r exported_model.zip eagleview/checkpoints/exported_model/
from google.colab import files
files.download("exported_model.zip")

  adding: eagleview/checkpoints/exported_model/ (stored 0%)
  adding: eagleview/checkpoints/exported_model/saved_model.pb (deflated 89%)
  adding: eagleview/checkpoints/exported_model/efficientdet-d2_frozen.pb (deflated 13%)
  adding: eagleview/checkpoints/exported_model/variables/ (stored 0%)
  adding: eagleview/checkpoints/exported_model/variables/variables.index (deflated 78%)
  adding: eagleview/checkpoints/exported_model/variables/variables.data-00000-of-00001 (deflated 8%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [135]:
X_train.to_csv('./eagleview/data/train.csv', index=False)

In [136]:
X_valid.to_csv('./eagleview/data/valid.csv', index=False)
X_test.to_csv('./eagleview/data/test.csv', index=False)

In [138]:
%cd eagleview/
!git add scripts
!git add config
!git add data/train.csv
!git add data/valid.csv
!git add data/test.csv
!git add notebooks
!git add logs
!git add results

/content/eagleview
hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> scripts
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached scripts
hint: 
hint: See "git help submodule" for more information.


In [142]:
!git commit -m "Sudhanshu | Adding Training Scripts and Results"

[main d44eca0] Harveen | Adding Training Scripts and Results
 207 files changed, 22358 insertions(+)
 create mode 100644 config/config.yaml
 create mode 100644 data/test.csv
 create mode 100644 data/train.csv
 create mode 100644 data/valid.csv
 create mode 100644 logs/train_log.out
 create mode 100644 results/image_000000020.jpg
 create mode 100644 results/image_000000024.jpg
 create mode 100644 results/image_000000028.jpg
 create mode 100644 results/image_000000065.jpg
 create mode 100644 results/image_000000082.jpg
 create mode 100644 results/image_000000084.jpg
 create mode 100644 results/image_000000113.jpg
 create mode 100644 results/image_000000128.jpg
 create mode 100644 results/image_000000130.jpg
 create mode 100644 results/image_000000132.jpg
 create mode 100644 results/image_000000151.jpg
 create mode 100644 results/image_000000158.jpg
 create mode 100644 results/image_000000159.jpg
 create mode 100644 results/image_000000182.jpg
 create mode 100644 results/image_000000195.j

In [154]:
%cd ../
!zip -r eagleview_code.zip eagleview
from google.colab import files
files.download("eagleview_code.zip")

/content
  adding: eagleview/ (stored 0%)
  adding: eagleview/notebooks/ (stored 0%)
  adding: eagleview/results/ (stored 0%)
  adding: eagleview/results/image_000000990.jpg (deflated 0%)
  adding: eagleview/results/image_000001017.jpg (deflated 1%)
  adding: eagleview/results/image_000001837.jpg (deflated 0%)
  adding: eagleview/results/image_000001984.jpg (deflated 3%)
  adding: eagleview/results/image_000000151.jpg (deflated 0%)
  adding: eagleview/results/image_000001255.jpg (deflated 0%)
  adding: eagleview/results/image_000001012.jpg (deflated 0%)
  adding: eagleview/results/image_000001763.jpg (deflated 0%)
  adding: eagleview/results/image_000000801.jpg (deflated 1%)
  adding: eagleview/results/image_000001139.jpg (deflated 2%)
  adding: eagleview/results/image_000000941.jpg (deflated 1%)
  adding: eagleview/results/image_000002024.jpg (deflated 0%)
  adding: eagleview/results/image_000001396.jpg (deflated 0%)
  adding: eagleview/results/image_000000666.jpg (deflated 1%)
  addi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>